In [78]:
import mwxml
import regex as re
from tqdm import tqdm

dump = mwxml.Dump.from_file(open("../data/dewiki-20220520-pages-articles-multistream1.xml-p1p297012"))
print(dump.site_info.name, dump.site_info.dbname)

Wikipedia dewiki


In [15]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

#lines = []
with open("../data/title-categories-map.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title + ","
        for revision in page:
            for match in pattern.finditer(revision.text):
                line += match.group(1) + ","
        line = line.strip(",")
        #lines.append(line)
        if words_re.search(str(line)):
            outfile.write(line+"\n")

155401it [00:40, 3855.27it/s]


In [17]:
num_lines = sum(1 for line in open('../data/title-categories-map.txt'))
print(num_lines)
#Output 1579

1579


Aufgabe 2c) 

1. Infobox der Unternehmen extrahieren

Der Aufbau einer Infobox eines Unternehmens in dessen Wikipedia Artikel ist streng definiert. Die Infobox beginnt mit zwei geschweiften Klammern und dem Typ des Eintrags, in diesem Fall {{{Infobox Unternehmen. Sie enthält maximal 14 Einträge: Name, Logo, Unternehmensform, ISIN, Gründungsdatum, Auflösungsdatum, Sitz, Leitung, Mitarbeiterzahl, Umsatz, Stand, Branche, Website. Erforderlich sind hiervon Unternehmensform sowie Sitz. 

Die Infobox wird mit folgendem Regex Pattern definiert: ((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}})).
Es wird der Inhalt nach {{Infobox Unternehmen bis einschließlich dem letzten Eintrag Homepage.*?}} ausgelesen. 

Verbesserungsvorschlag: Einträge einzeln auslesen, sodass nur die Parameter ausgegeben werden, die einen Wert besitzen.

In [36]:
#getting the infobox of a company

infobox = re.compile('((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}}))') #regex pattern gets text between keyword 'Infobox Unternehmen' and 'Homepage' 

with open("../data/infobox.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ''
        for revision in page: 
            revision = re.sub(r'\n', '', revision.text) #removing linebreaks 
            for match in infobox.finditer(revision):
                line += match.group(1) + "\n" 
                line = " ".join(line.split()) #if there are multiple whitespaces, all except for one get deleted 
                outfile.write(line+"\n")
    

155401it [02:41, 959.30it/s] 


In [37]:
num_lines = sum(1 for line in open('../data/infobox.txt'))
print(num_lines)
#Output:697

697


Nun soll auch für Unternehmen ohne Infobox deren Namen sowie dessen Webseite ausgegeben werden. 

Hierfür wird im ersten Schritt geprüft, ob der Wikipedia Artikel des Unternehmens das Wort Infobox enthält. Dies wird mit folgendem Regex Pattern geprüft: (?!.*?Infobox)^.*$ 
Enthält der Artikel das Wort nicht, so erkennt das Regex Pattern den gesamten Artikel. 
Wird der Artikel als Unternehmensartikel klassifiziert und enthält er das Wort Infobox nicht, wird der Artikelname (=Unternehmensname) ausgegeben. 

Um nun auch die Webseite des Unternehmens ausgeben zu lassen, wird folgendes Regex Pattern angewandt: 
.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*
Dies gibt alle in der Sektion Weblinks eingetragnenen Webseiten aus, worunter oftmals auch die offizielle Unternehmenswebseite fällt. 

Verbesserungsvorschlag: Es werden viele Artikel ausgelesen, die die Kategorie Unternehmsart enthalten, aber keine Unternehmen sind. Diese sind oftmals allgemeine Informationsseiten und enthalten daher auch keine Infobox, weshalb sie vermehrt ausgelesen werden. 
Darüber hinaus wird für die Unternehmenswebseite die ganze Sektion der Weblinks ausgelesen, die oftmals auch andere weiterführende Links enthält. Eine Verbesserungsmöglichkeit bestände darin, den Namen des Unternehmens in den Links zu suchen und nur die Links, die den Namen enthalten, auszugegeben. 

In [40]:
#Companies without infobox
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

no_infobox = re.compile('(?!.*?Infobox)^.*$') #gets the whole article if it doesn´t contain 'Infobox'
pattern_website = re.compile('.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*') 


with open("../data/company_name&website.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title
        website = ""
        for revision in page:
            revision = re.sub(r'\n', '', revision.text) #removing the line breaks 
            for match in pattern.finditer(revision):
                line += match.group(1)  
        
        if words_re.search(str(line)):
            for match in no_infobox.finditer(revision): #getting companies without an infobox 
                for match in pattern_website.finditer(revision): #getting the weblinks which contain the company url 
                    outfile.write(page.title+"\n") #printing the page title = company name 
                    website += match.group(1)
                    outfile.write(website+"\n"+"\n")

155401it [00:48, 3185.01it/s]


In [ ]:
companies_without_infobox = sum((1/3) for line in open('../data/company_name&website.txt'))
print(companies_without_infobox)
#Output: 136 

Um nun eine Kurzbeschreibung des Unternehmens zu erhalten, wird der erste Satz des Artikels erfasst. Dieser enthält bei Unternehmen den Unternehmensnamen eingefasst in ''' '''. Aus diesem Grund wird folgendes Regex Pattern angewandt, um den ersten Satz auszulesen: 
((?<=\''' ).*?(?<=\.)+) 

In [87]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

pattern_firstsentence = re.compile('((?<=\''' ).*?(?<=\.))')
#Erster Satz fängt meistens mit dem Firmennamen in ''' ''' an. Dann wird der erste Punkt gesucht.

with open("../data/firstsentence.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title
        firstsentence = ""
        for revision in page:
            for match in pattern.finditer(revision.text):
                line += match.group(1)  
        
        if words_re.search(str(line)):
            for match in pattern_firstsentence.finditer(revision.text):
                firstsentence += match.group(1)
                firstsentence.replace(r'\*', '')
            outfile.write(page.title+"\n")
            outfile.write(firstsentence+"\n"+"\n")

154157it [00:41, 3682.78it/s]


In [ ]:
firstsentences = sum((1/3) for line in open('../data/firstsentence.txt'))
print(firstsentences)
#Output: 1570